# Aprendizaje automático para datos genómicos: predicción de bacterias resistentes a antibióticos

Bienvenido a la segunda sesión del **Taller**. Hoy, construiremos modelos
de predicción de resistencia usando datos de tres fuentes diferentes:

1. Genes con funciones conocidas.
2. Genes con funciones conocidas y desconocidas.
3. SNPs o mutaciones puntuales.

¿Cuál será el mejor?

## Resumen de la sesión anterior

En la sesión anterior, aprendimos cómo:

1. Precargar lecturas de NCBI SRA con `prefetch`.
2. Extraer lecturas precargadas con `fasterq-dump`.
3. Ensamblar genomas con `spades.py`.
4. Anotar genomas con `prokka`.
5. Extraer proteínas hipotéticas con `seqtk`.

La tarea fue repetir todos estos pasos sobre todas las muestras.

Una posible solución se muestra a continuación (**ALERTA**: la ejecución es
tardada).

In [ ]:
%%bash

# Crear carpetas para datos
mkdir data
mkdir data/reads data/genomes data/annotations

# Cada fila de metadata.tsv contiene cuatro valores separados por tabulador
while read -r sample read mic phenotype; do

  # 1. Precargar lecturas
  prefetch -O data/reads ${read}

  # 2. Extraer lecturas
  fasterq-dump -O data/reads data/reads/${read}

  # 3. Ensamblar lecturas en contigs
  spades.py --isolate \
    -1 data/reads/${read}_1.fastq \
    -2 data/reads/${read}_2.fastq \
    -o data/genomes/${sample}

  # 4. Anotar el genoma ensamblado
  prokka --outdir data/annotations/${sample} \
    --prefix ${sample} \
    --locustag ${sample} \
    --genus Mycoplasmoides \
    --species genitalium \
    --strain ${sample} \
    --mincontiglen 1000 \
    data/genomes/${sample}/contigs.fasta

  # 5. Extraer las proteínas hipotéticas a un FASTA aparte
  grep 'hypothetical protein' data/annotations/${sample}/${sample}.tsv | \
    cut -f 1 | \
    seqtk subseq data/annotations/${sample}/${sample}.faa - \
    > data/annotations/${sample}/${sample}.hypothetical.faa

done < metadata.tsv

En lugar de correr la celda anterior, descarga los resultados precalculados.

In [ ]:
!wget 'https://github.com/aapashkov/biotaller2025/releases/latest/download/preprocessing.zip'
!unzip 'preprocessing.zip'

## Intento 1: usando funciones para clasificar resistencia

Necesitaremos matplotlib para poder hacer visualizaciones.

In [ ]:
%pip install -q matplotlib

Prokka nos proporciona los nombres de genes con funciones conocidas.

Una primera idea para crear un modelo de predicción sería utilizar estas
funciones como atributos.

Primero, carguemos las etiquetas que deseamos predecir, es decir, los fenotipos
de "resistente" y "susceptible".

Codificaremos "resistente" como 1 y "susceptible" como 0.

Ahora cargaremos las funciones predichas por Prokka.

Para ello, leeremos cada archivo TSV de Prokka y creamos una tabla donde la
primera columna almacena la muestra y la segunda guarda las funciones de la
muestra.

Ahora, para transformar estos datos en algo que un modelo pueda utilizar para
entrenar, realizaremos una tabulación cruzada de la tabla de funciones.

Antes de crear nuestro modelo, vamos a evaluar la importancia de cada función
respecto al fenotipo utilizando la información mutua.

Visualizamos las 20 funciones más importantes.

Podemos crear ahora un modelo de clasificación.

Usaremos las 6 variables más importantes.

El modelo está teniendo dificultades para poder clasificar.

¿Y si agregamos las proteínas con funciones desconocidas?

## Intento 2: agregando funciones desconocidas mediante homología

¿Cómo podemos agrupar proteínas por categoría si desconocemos sus funciones?

El secreto es el parentesco (**homología**) entre las secuencias:

- **Genes homólogos**: par de genes con **secuencia similar** y que muy
probablemente poseen la **misma función**.
- **Genes ortólogos**: par de genes homólogos presentes en **genomas diferentes**.
- **Genes parálogos**: par de genes homólogos presentes en el **mismo genoma**.

**Proteinortho** es un programa que identifica proteínas ortólogas y parálogas
a partir de un conjunto de proteínas de entrada.

In [ ]:
%%bash

# Crear carpeta de salida para ortólogos de proteínas con funciones desconocidas
mkdir data/orthologs
cd data/orthologs

# Corremos Proteinortho sobre las proteínas hipotéticas
proteinortho ../annotations/*/*.hypothetical.faa

Cargamos la table de proteínas hipotéticas y la ajustamos a un formato análogo
al de la tabla de funciones.

Pegamos esta tabla a la tabla de funciones.

Veamos las variables más importantes.

Probemos un modelo con las 20 variables más importantes.

¿Qué tal si probamos trabajar con mutaciones en lugar de genes o proteínas?

## Intento 3: modelo basado en mutaciones o SNPs

Para crear un modelo basado en SNPs, primero necesitamos crear un FASTA por cada
función única.

Para eso, creamos un archivo con la lista de funciones.

Creamos los FASTAs con seqtk. Por cada función, habrá un archivo llamado
`data/snps/funcion.unaligned.fasta`.

In [ ]:
%%bash
mkdir -p data/snps

while read -r function; do

  filename=data/snps/$(tr '/' '_' <<< "${function}" | tr ' ' '_').unaligned.fasta

  cat data/annotations/*/*.tsv | \
    grep -F "${function}" | \
    cut -f 1 | \
    seqtk subseq <(cat data/annotations/*/*.ffn) - \
    > "${filename}"

done < data/functions.txt

Usamos MAFFT para alinear los genes de cada función.

In [ ]:
%%bash

for unaligned in data/snps/*.unaligned.fasta; do
  function=$(basename "${unaligned}" .unaligned.fasta)
  aligned="data/snps/${function}.aligned.fasta"
  mafft "${unaligned}" > "${aligned}" 2> /dev/null
done

Finalmente, usamos snp-sites para identificar los SNPs.

In [ ]:
%%bash
for aligned in data/snps/*.aligned.fasta; do
  function=$(basename "${aligned}" .aligned.fasta)
  snps="data/snps/${function}.vcf"
  snp-sites -v -c "${aligned}" -o "${snps}"
done

Vamos a crear una función que cargue un archivo VCF en un formato compatible para entrenamiento.

Usaremos la función `glob` para cargar todos los archivos VCF en una tabla.

Veamos las 20 variables más importantes.

Veamos el rendimiento del modelo usando las 20 mejores variables.

Probemos algo diferente: ¿qué tal si en lugar de pasarle todos los SNPs
de todos los genes, entrenamos un modelo con los SNPs de cada gen?

Tabulamos en una tabla y obtenemos los mejores modelos.

---

## ¿Dónde seguir aprendiendo/practicando?

- Talleres de The Carpentries:
    - Pangenómica: https://carpentries-incubator.github.io/pangenomics-workshop/
    - Minería Genómica: https://carpentries-incubator.github.io/genome-mining/
    - Metagenómica: https://carpentries-lab.github.io/metagenomics-workshop/
- Reto CAMDA 2025, incluye concurso de predicción de bacterias resistentes (>6000 genomas): https://bipress.boku.ac.at/camda2025/